# NLE Assessed Coursework 3: Question 1

For this assessment, you are expected to complete and submit 4 notebook files.  There is 1 notebook file for each question (to speed up load times).  This is notebook 1 out of 4.

Marking guidelines are provided as a separate document.

In order to provide unique datasets for analysis by different students, you must enter your candidate number in the following cell.

In [1]:
candidateno=215939 #this MUST be updated to your candidate number so that you get a unique data sample


In [2]:
#preliminary imports
import sys
#sys.path.append(r'\\ad.susx.ac.uk\ITS\TeachingResources\Departments\Informatics\LanguageEngineering\resources')
#sys.path.append(r'/Users/juliewe/resources')
sys.path.append('.')
sys.path.append(r"./resources")
import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import zip_longest
from nltk.tokenize import word_tokenize
import nltk
nltk.download('gutenberg')
from sussex_nltk.corpus_readers import AmazonReviewCorpusReader
#import corpus_readers
import random
from nltk.corpus import stopwords
from nltk.tag import UnigramTagger
import spacy
import en_core_web_sm
nlp=en_core_web_sm.load()
from nltk.corpus import gutenberg
from types import MethodType

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Panda\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


Sussex NLTK root directory is ./resources


## Question 1: Part-of-Speech Tagging (25 marks)


In [3]:
#Do NOT change the code in this cell.

#preparing corpus
def display_sent(asent):
    headings=["token","lower","lemma","pos","NER"]
    info=[]
    for t in asent:
        info.append([t.text,t.lower_,t.lemma_,t.pos_,t.ent_type_])
    return(pd.DataFrame(info,columns=headings))

def tag_sent(asent):
    tagged=[]
    for t in asent:
        tagged.append((t.lower_,t.pos_))
    return tagged

def clean_text(astring):
    #replace newlines with space
    newstring=re.sub("\n"," ",astring)
    #remove title and chapter headings
    newstring=re.sub("\[[^\]]*\]"," ",newstring)
    newstring=re.sub("VOLUME \S+"," ",newstring)
    newstring=re.sub("CHAPTER \S+"," ",newstring)
    newstring=re.sub("\s\s+"," ",newstring)
    #return re.sub("([^\.|^ ])  +",r"\1 .  ",newstring).lstrip().rstrip()
    return newstring.lstrip().rstrip()

def display_tags(sentslist):
    taglist={}
    for s in sentslist:
        for t in s:
            form=t.lower_
            pos=t.pos_
            taglist[pos]=taglist.get(pos,0)+1
    print(len(taglist.keys()))
    print(taglist)
        
def get_train_test(sentslist,seed=candidateno):
    random.seed(seed)
    random.shuffle(sentslist)
    testsize=10
    train=[tag_sent(s) for s in sentslist[testsize:]]
    test=[tag_sent(s) for s in sentslist[:testsize]]
    return train,test
    
alice=clean_text(gutenberg.raw('carroll-alice.txt'))

nlp_alice=list(nlp(alice).sents)
print(nlp_alice)

[Down the Rabbit-Hole Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?', So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her., There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself, 'Oh dear!, Oh dear!, I shall be late!', (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the Rabbit actually TOOK A WATCH OUT OF ITS WAISTCOAT-POCKET

The code below will generate (unique to you) training and test sets of pos-tagged sentences from the novel "Alice in Wonderland"

In [4]:
#do not change the code in this cell
allsents=list(nlp_alice)
train,test=get_train_test(allsents)

In the cell below, a class `unigram_tagger` is defined, which will be used in parts a and b of this question

In [5]:
#do not change the code in this cell
class unigram_tagger():
    
    def __init__(self,trainingdata=[]):
        self.tagcounts={}
        self.wordcounts={}
        self.tagperwordcounts={}
        self.train(trainingdata=trainingdata)
        
    def train(self,trainingdata):
        
        for sentence in trainingdata:
            for token,tag in sentence:
                self.tagcounts[tag]=self.tagcounts.get(tag,0)+1
                self.wordcounts[token]=self.wordcounts.get(token,0)+1
                current=self.tagperwordcounts.get(token,{})
                current[tag]=current.get(tag,0)+1
                self.tagperwordcounts[token]=current
                

a) Create an instance of the `unigram_tagger` class which is trained on your training sentences from "Alice in Wonderland" (stored in `train`).
Explain what is stored, after training, in the three instance variables:
* `self.tagcounts`
* `self.wordcounts`
* `self.tagperwordcounts`

You should refer to the code and specific examples from the training data. \[4 marks \]

In [6]:
uni_tagger = unigram_tagger(train)


    

In [7]:
print(uni_tagger.tagcounts)
print(uni_tagger.wordcounts)
print(uni_tagger.tagperwordcounts)

{'PRON': 3217, 'VERB': 4607, 'PART': 847, 'AUX': 1624, 'PUNCT': 7148, 'ADP': 2593, 'DET': 3538, 'NOUN': 3295, 'CCONJ': 1112, 'ADV': 2268, 'PROPN': 1487, 'ADJ': 1383, 'SCONJ': 669, 'NUM': 200, 'INTJ': 149, 'X': 7}
{'you': 404, 'sha': 6, "n't": 213, 'be': 146, 'beheaded': 3, '!': 447, "'": 2198, 'they': 151, 'are': 51, 'waiting': 9, 'on': 191, 'the': 1615, 'shingle': 1, '--': 132, 'will': 32, 'come': 45, 'and': 858, 'join': 9, 'dance': 13, '?': 196, 'so': 149, 'see': 67, ',': 2389, 'miss': 4, 'we': 34, "'re": 38, 'doing': 5, 'our': 8, 'best': 12, 'i': 532, 'would': 96, 'in': 364, 'bill': 16, "'s": 197, 'place': 8, 'for': 151, 'a': 625, 'good': 27, 'deal': 12, ':': 216, 'this': 132, 'fireplace': 1, 'is': 107, 'narrow': 2, 'to': 725, 'sure': 24, ';': 191, 'but': 164, 'think': 49, 'can': 34, 'kick': 3, 'little': 126, 'do': 138, 'said': 454, 'caterpillar': 26, '.': 963, 'down': 101, 'she': 551, 'came': 40, 'upon': 26, 'heap': 1, 'of': 510, 'sticks': 1, 'dry': 7, 'leaves': 6, 'fall': 7, 'was'

In [ ]:
#tagcounts means the number of tags in the sentence
#wordcounts means the number of tokens in the sentence
#tagperwordcounts represent the tokens,the tags corresponding to that tokens and the number of the tags


b) In the cells below, **extend** the code for the `unigram_tagger` class so that it also has a `tag()` method.  This method should assign the tag, $t$, which maximises the unigram tag probability, $P(t|w)$, for the observed word, $w$.  **Evaluate** the performance of the `unigram_tagger` on your test sentences.  **Discuss** your results. \[8 marks\] 



In [6]:
def tag(tokens):
    # docs inherited from TaggerI
    tags = []
   
    for i in range(len(tokens)):
        tags.append(tag_one(tokens, i, tags))
    return list(zip(tokens, tags))
def tag_one(tokens, index, history):
    tag = None
    print(tokens)
    for token,tagger in uni_tagger.tagperwordcounts:
        if tokens[index] in token:
             tagger_index = token.index(tokens[index])
             for key,value in  tagger[tagger_index]:
                 tag = key[0]
    return tag
   
print(tag(train))



[[('you', 'PRON'), ('sha', 'VERB'), ("n't", 'PART'), ('be', 'AUX'), ('beheaded', 'VERB'), ('!', 'PUNCT'), ("'", 'PUNCT')], [('they', 'PRON'), ('are', 'AUX'), ('waiting', 'VERB'), ('on', 'ADP'), ('the', 'DET'), ('shingle', 'NOUN'), ('--', 'PUNCT'), ('will', 'VERB'), ('you', 'PRON'), ('come', 'VERB'), ('and', 'CCONJ'), ('join', 'VERB'), ('the', 'DET'), ('dance', 'NOUN'), ('?', 'PUNCT')], [("'", 'PUNCT')], [('so', 'ADV'), ('you', 'PRON'), ('see', 'VERB'), (',', 'PUNCT'), ('miss', 'PROPN'), (',', 'PUNCT'), ('we', 'PRON'), ("'re", 'AUX'), ('doing', 'VERB'), ('our', 'DET'), ('best', 'ADJ'), (',', 'PUNCT')], [('i', 'PRON'), ('would', 'VERB'), ("n't", 'PART'), ('be', 'AUX'), ('in', 'ADP'), ('bill', 'PROPN'), ("'s", 'PART'), ('place', 'NOUN'), ('for', 'ADP'), ('a', 'DET'), ('good', 'ADJ'), ('deal', 'NOUN'), (':', 'PUNCT'), ('this', 'DET'), ('fireplace', 'NOUN'), ('is', 'AUX'), ('narrow', 'ADJ'), (',', 'PUNCT'), ('to', 'PART'), ('be', 'AUX'), ('sure', 'ADJ'), (';', 'PUNCT'), ('but', 'CCONJ'), ('

NameError: name 'uni_tagger' is not defined

In the cell below, a `hmm_tagger` class is defined, which will be used in parts c and d of this question.

In [7]:
#do not change the code in this cell
class hmm_tagger():
    
    def __init__(self,trainingdata=[]):
        
        self.emissions={}
        self.transitions={}
        self.train(trainingdata=trainingdata)
        
    def train(self,trainingdata):
        
        for sentence in trainingdata:
            previous="START"
            for token,tag in sentence:
                
                current=self.emissions.get(tag,{})
                current[token]=current.get(token,0)+1
                self.emissions[tag]=current
                bigram=self.transitions.get(previous,{})
                bigram[tag]=bigram.get(tag,0)+1
                self.transitions[previous]=bigram
                previous=tag
                
        self.emissions={tag:{token:freq/sum(countdict.values()) for (token,freq) in countdict.items()}for (tag,countdict) in self.emissions.items()}
        self.transitions={tag:{token:freq/sum(countdict.values()) for (token,freq) in countdict.items()}for (tag,countdict) in self.transitions.items()}

c) **Create** an instance of the `hmm_tagger` class which is trained on your training sentences from "Alice in Wonderland" (stored in `train`).  With reference to your testing sentences (stored in `test`), **explain**
* how to calculate the probability of an observed sequence of words for a given sequence of tags: $$P(w_1^n|t_1^n)$$
* how to calculate the probablity of a possible sequence of tags for a given sequence of words: $$P(t_1^n|w_1^n)$$

\[6 marks\]

In [8]:
hmm =  hmm_tagger(train)


In [9]:
emission = hmm.emissions
transition =  hmm.transitions
print(emission)
print(transition)

{'PRON': {'you': 0.12558284115635684, 'they': 0.04693814112527199, 'we': 0.010568852968604289, 'i': 0.16537146409698478, 'she': 0.17034504196456327, 'there': 0.027043829654958034, 'what': 0.04165371464096985, 'it': 0.18277898663350947, 'me': 0.021137705937208578, 'who': 0.019583462853590302, 'anything': 0.006216972334473112, 'he': 0.038234379857009636, 'her': 0.027976375505129002, 'them': 0.026422132421510723, 'us': 0.004351880634131178, 'himself': 0.0018650917003419335, 'him': 0.01336649051911719, 'herself': 0.025800435188063412, "'s": 0.001554243083618278, 'everybody': 0.0018650917003419335, 'myself': 0.0018650917003419335, 'yourself': 0.003108486167236556, 'nobody': 0.002175940317065589, 'whom': 0.00031084861672365556, 'nothing': 0.010258004351880633, 'somebody': 0.0018650917003419335, 'one': 0.002175940317065589, 'mine': 0.0009325458501709667, 'be"--or': 0.00031084861672365556, 'everything': 0.004351880634131178, 'itself': 0.004351880634131178, 'ourselves': 0.00031084861672365556, 

d) Using one of your `test` sentences as an example, **explain** how the Viterbi algorithm can be used to find the most likely tag sequence.  You do not need to write code for this question but should explain the calculations that need to be made at each step.  **Comment** on whether the sequence found by the Viterbi algorithm is correct for your chosen test sentence.  **Discuss** why using the Viterbi algorithm is better than the brute-force approach of enumerating and testing all tag sequence possibilities.\[7 marks\]
